In [ ]:
import torch
import torch.nn as nn
import numpy as np
import nltk
import json
import random
from nltk.stem.porter import PorterStemmer
from torch.utils.data import Dataset, DataLoader

# Download nltk data
nltk.download('punkt')

# Initialize stemmer
stemmer = PorterStemmer()

# Tokenization function
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

# Stemming function
def stem(word):
    return stemmer.stem(word.lower())

# Bag of words function
def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag

# Load intents file
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

# Process each intent in the intents file
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# Stemming and removing duplicates
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# Training data preparation
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyperparameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# Define dataset
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# DataLoader
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Define the neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

# Save the model
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'Training complete. File saved to {FILE}')


FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "MovieBot"
print("Let's chat! (type 'quit' to exit)")

# Genre keywords dictionary
genre_keywords = {
    "action": "Action",
    "horror": "Horror",
    "thriller": "Thriller",
    "western": "Western",
    "drama": "Drama",
    "fantasy": "Fantasy",
    "adventure": "Adventure",
    "romance": "Romance",
    "comedy": "Comedy",
    "mystery": "Mystery",
    "sci-fi": "Sci-Fi",
    "animation": "Animation",
    "supernatural": "Supernatural",
    "religious": "Religious",
    "biography": "Biography",
    "suspense": "Suspense"
}

while True:
    sentence = input("You: ")
    if sentence.lower() == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                response = random.choice(intent['responses'])
                if '{movie_title}' in response:
                    # Detect genre preference from the user's query
                    detected_genre = None
                    for word in sentence:
                        word_stemmed = stem(word)
                        if word_stemmed in genre_keywords:
                            detected_genre = genre_keywords[word_stemmed]
                            break

                    # If a genre is detected, choose a movie from that genre
                    if detected_genre and detected_genre in intents['genres']:
                        movie = random.choice(intents['genres'][detected_genre])
                        response = response.replace('{movie_title}', f"{movie['title']} ({movie['language']})")
                    else:
                        genre = random.choice(list(intents['genres'].keys()))
                        movie = random.choice(intents['genres'][genre])
                        response = response.replace('{movie_title}', f"{movie['title']} ({movie['language']})")
                print(f"{bot_name}: {response}")
    else:
        print(f"{bot_name}: I do not understand...")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch [100/1000], Loss: 1.2131
Epoch [200/1000], Loss: 0.1283
Epoch [300/1000], Loss: 0.0367
Epoch [400/1000], Loss: 0.0176
Epoch [500/1000], Loss: 0.0069
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0012
Epoch [800/1000], Loss: 0.0007
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0003
Final loss: 0.0003
Training complete. File saved to data.pth
Let's chat! (type 'quit' to exit)
You: hi
MovieBot: Hi there, what can I do for you?
You: hello
MovieBot: Hi there, what can I do for you?
You: what is your name
MovieBot: I'm a Moviebot, so I don't have a personal name.
You: what is your age
MovieBot: I exist outside the concept of age!
You: what movie do you lik
MovieBot: I'm known as a chatbot, no personal name here!
You: what kind of movie do you like
MovieBot: I can recommend movies based on your preferred genres. Let me know what you like!
You: suggest me action movies
MovieBot: Consider watching Simmba (Hindi). It's great!
You: can you suggest me horror movies
Mov